In [5]:
# Necessary impports and pip installs
!pip3 install googlesearch-python
import pandas as pd
import os
import numpy as np
import re
from googlesearch import search
import requests
from bs4 import BeautifulSoup

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.9/bin/python3.9 -m pip install --upgrade pip' command.


In [6]:
'''
Dataframe loading. 
drone_df : raw FAA drone intrusion info.
airport_codes_2 : huge dataset with info on airport codes, latitude & longitude
'''

path_df = "fy24_q2.xlsx"
drone_df = pd.read_excel(path_df)

path_3 = "all-airport-data.xlsx"
airport_codes_2 = pd.read_excel(path_3)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [7]:
''''
Line of code to extract all the 3 letter airport codes. 
'''

list_codes = list(airport_codes_2["Loc Id"])

drone_df['code'] = None

for index_2, row in drone_df.iterrows():
  x = row["Summary"]
  splitted_x = x.split()
  punc = '''!()-[]{};:'"\,<>.,/?@#$%^&*_~'''

  # Removing punctuations in string
  # Using loop + punctuation string
  index = 0
  for word in splitted_x:
      for letter in word:
        if letter in punc:
          word = word.replace(letter, "")
          splitted_x[index] = word
      index = index + 1
  filtered_codes = list(set([i for i in splitted_x if len(i) == 3 and i not in ["FAA", "UAS", "OPS", "THE"]]))
  for i in filtered_codes:
    if i in list_codes:
       drone_df.at[index_2, 'code'] = i

drone_df

,Date,State,City,Summary,code
0,2024-01-01,FLORIDA,MIAMI,"PRELIM INFO FROM FAA OPS: MIAMI, FL/UAS SIGHTI...",MIA
1,2024-01-01,TEXAS,FORT WORTH,"PRELIM INFO FROM FAA OPS: FORT WORTH, TX/UAS S...",FTW
2,2024-01-01,THE BAHAMAS,BAHAMAS,"PRELIM INFO FROM FAA OPS: BIMINI, BAHAMAS/UAS ...",FLL
3,2024-01-01,FLORIDA,MIAMI,"PRELIM INFO FROM FAA OPS: MIAMI, FL/UAS SIGHTI...",HPN
4,2024-01-01,COLORADO,DENVER,"PRELIM INFO FROM FAA OPS: DENVER, CO/UAS SIGHT...",DEN
...,...,...,...,...,...
321,2024-03-15,PR,MAYAGUEZ,"PRELIM INFO FROM FAA OPS: MAYAGUEZ, PUERTO RIC...",OFF
322,2024-03-31,GEORGIA,ATLANTA,"PRELIM INFO FROM FAA OPS: ATLANTA, GA/UAS SIGH...",MYR
323,2024-03-31,CALIFORNIA,FRESNO,"PRELIM INFO FROM FAA OPS: FRESNO, CA/UAS SIGHT...",FAT
324,2024-03-31,NEW JERSEY,CALDWELL,"PRELIM INFO FROM FAA OPS: CALDWELL, NJ/UAS SIG...",CDW


In [8]:
'''
All regex commands for drone information extraction.
'''

def extract_direction(summary):
    match = re.search(r'WHILE\s([A-Z]+\sBOUND)', summary)
    if match:
        return match.group(1)
    else:
        return np.nan

drone_df['Direction'] = drone_df['Summary'].apply(extract_direction)

pattern_time = r'(?<=\/)([0-9]{4}[A-Z])'
drone_df['Time'] = drone_df['Summary'].str.extract(pattern_time)

pattern_advised = r'(?<=\/)([A-Z ]+)(?= ADVISED)'
drone_df['Authority Advised'] = drone_df['Summary'].str.extract(pattern_advised)

pattern_route = r'(?<=, )([A-Z  ,()]+ - [A-Z  ,()]+)(?=,)'
drone_df['Route'] = drone_df['Summary'].str.extract(pattern_route)

pattern_location = r'(?<= )([0-9]+ [(W|ESE|E|N|NW|SW|S|SE|NE|NNE|NNW|WNW|WSW|NORTHWEST|ENE|SSE|SSW)]+ [A-Z() ]+)(?= AT| FEET|[.])'
drone_df['Location'] = drone_df['Summary'].str.extract(pattern_location)

pattern_altitude = r'(?<= )(AT [0-9,]+ FEET)(?= )'
drone_df['Altitude'] = drone_df['Summary'].str.extract(pattern_altitude)

drone_df

,Date,State,City,Summary,code,Direction,Time,Authority Advised,Route,Location,Altitude
0,2024-01-01,FLORIDA,MIAMI,"PRELIM INFO FROM FAA OPS: MIAMI, FL/UAS SIGHTI...",MIA,W BOUND,0001E,MIAMI ATCT,"PANAMA CITY, PANAMA (MPTO) - MIA",1 E MIA,AT 500 FEET
1,2024-01-01,TEXAS,FORT WORTH,"PRELIM INFO FROM FAA OPS: FORT WORTH, TX/UAS S...",FTW,ESE BOUND,0015C,MEACHAM ATCT,FTW - FTW,8 ESE FTW,"AT 1,900 FEET"
2,2024-01-01,THE BAHAMAS,BAHAMAS,"PRELIM INFO FROM FAA OPS: BIMINI, BAHAMAS/UAS ...",FLL,E BOUND,0655E,MIAMI ARTCC,"FLL - GOVERNORS HARBOUR, BAHAMAS (MYEN)",7 E BIMINI VORTAC,"AT 9,000 FEET"
3,2024-01-01,FLORIDA,MIAMI,"PRELIM INFO FROM FAA OPS: MIAMI, FL/UAS SIGHTI...",HPN,NaN,1010E,MIA,MIA - HPN,4 W MIA,NaN
4,2024-01-01,COLORADO,DENVER,"PRELIM INFO FROM FAA OPS: DENVER, CO/UAS SIGHT...",DEN,N BOUND,0945M,DEN ATCT,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
321,2024-03-15,PR,MAYAGUEZ,"PRELIM INFO FROM FAA OPS: MAYAGUEZ, PUERTO RIC...",OFF,E BOUND,1143L,SAN JUAN CERAP,"LA ROMANA, DOMINICAN REPUBLIC (MDLR) - SJU",10 S MAYAGUEZ (TJMZ),"AT 9,000 FEET"
322,2024-03-31,GEORGIA,ATLANTA,"PRELIM INFO FROM FAA OPS: ATLANTA, GA/UAS SIGH...",MYR,N BOUND,1210E,ATLANTA TRACON,MYR - ATL,20 SE ATL,"AT 5,000 FEET"
323,2024-03-31,CALIFORNIA,FRESNO,"PRELIM INFO FROM FAA OPS: FRESNO, CA/UAS SIGHT...",FAT,SE BOUND,0920P,FRESNO ATCT,CCR - VIS,10 WNW FAT,"AT 8,000 FEET"
324,2024-03-31,NEW JERSEY,CALDWELL,"PRELIM INFO FROM FAA OPS: CALDWELL, NJ/UAS SIG...",CDW,NaN,1156E,MCGUIRE APCH,CDW - UNKN,NaN,NaN


In [9]:
'''
Using Beautiful Soup and Google Search with Airnav in order to map the values for which the FAA code could not be found in original dataset.
'''

edge_df = drone_df[drone_df["code"].isna()]
edge_df_location = edge_df[~edge_df["Location"].isna()]

drone_df['latitude'] = None
drone_df['longitude'] = None

# Iterate over rows
for idx, location in edge_df_location.iterrows():
    loc_list = location['Location'].split()[2:]
    full_string = " ".join(loc_list)
    query_plain = " airnav site:airnav.com"
    query = full_string + query_plain

    # Perform Google search 
    search_results = search(query, num_results=1)  
    final_url = next(search_results, None)
    
    if final_url:
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36"
        }

        response = requests.get(final_url, headers=headers)

        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            
            faa_code = None
            latitude = None
            longitude = None
            
            table_rows = soup.find_all('tr')  
            for row in table_rows:
                if 'FAA Identifier' in row.text:
                    faa_code = row.find_all('td')[1].text.strip()  
                
                if 'Lat/Long' in row.text:
                    lat_long_text = row.find_all('td')[1].text.strip()
                    lat_long_split = lat_long_text.split("W")
                    
                    if len(lat_long_split) >= 2:
                        latitude = lat_long_split[0] + 'W' 
                        longitude = 'W' + lat_long_split[1]

            if not (latitude and longitude):
                string_soup = str(soup)
                result_re = re.findall(r'(?<=Lat\/Long:\s)([0-9\-\.]+[N|S])\s\/\s([0-9\-\.]+[E|W])', string_soup)
                if result_re:
                    latitude, longitude = result_re[0]

            if faa_code:
                drone_df.at[idx, 'code'] = faa_code
            if latitude and longitude:
                drone_df.at[idx, 'latitude'] = latitude
                drone_df.at[idx, 'longitude'] = longitude
        else:
            print(f"Failed to fetch the page for {location['Location']}, status code: {response.status_code}")

In [10]:
'''
Merging the results with the initial airport codes dataframe to get latitude and longitude for all rows.
'''

merged_drone_df = pd.merge(drone_df, airport_codes_2[['Loc Id', 'ARP Latitude DD', 'ARP Longitude DD']], 
                           left_on='code', right_on='Loc Id', how='left')

# Step 2: Only update 'latitude' and 'longitude' where they are NaN in drone_df
merged_drone_df['latitude'] = merged_drone_df['latitude'].fillna(merged_drone_df['ARP Latitude DD'])
merged_drone_df['longitude'] = merged_drone_df['longitude'].fillna(merged_drone_df['ARP Longitude DD'])

# Optional: Drop the 'ARP Latitude DD', 'ARP Longitude DD', and 'Loc Id' columns if you don't need them
merged_drone_df.drop(['Loc Id', 'ARP Latitude DD', 'ARP Longitude DD'], axis=1, inplace=True)
merged_drone_df

,Date,State,City,Summary,code,Direction,Time,Authority Advised,Route,Location,Altitude,latitude,longitude
0,2024-01-01,FLORIDA,MIAMI,"PRELIM INFO FROM FAA OPS: MIAMI, FL/UAS SIGHTI...",MIA,W BOUND,0001E,MIAMI ATCT,"PANAMA CITY, PANAMA (MPTO) - MIA",1 E MIA,AT 500 FEET,25.795361,-80.290116
1,2024-01-01,TEXAS,FORT WORTH,"PRELIM INFO FROM FAA OPS: FORT WORTH, TX/UAS S...",FTW,ESE BOUND,0015C,MEACHAM ATCT,FTW - FTW,8 ESE FTW,"AT 1,900 FEET",32.81977,-97.36245
2,2024-01-01,THE BAHAMAS,BAHAMAS,"PRELIM INFO FROM FAA OPS: BIMINI, BAHAMAS/UAS ...",FLL,E BOUND,0655E,MIAMI ARTCC,"FLL - GOVERNORS HARBOUR, BAHAMAS (MYEN)",7 E BIMINI VORTAC,"AT 9,000 FEET",26.071667,-80.149694
3,2024-01-01,FLORIDA,MIAMI,"PRELIM INFO FROM FAA OPS: MIAMI, FL/UAS SIGHTI...",HPN,NaN,1010E,MIA,MIA - HPN,4 W MIA,NaN,41.066953,-73.707566
4,2024-01-01,COLORADO,DENVER,"PRELIM INFO FROM FAA OPS: DENVER, CO/UAS SIGHT...",DEN,N BOUND,0945M,DEN ATCT,NaN,NaN,NaN,39.861667,-104.673167
...,...,...,...,...,...,...,...,...,...,...,...,...,...
321,2024-03-15,PR,MAYAGUEZ,"PRELIM INFO FROM FAA OPS: MAYAGUEZ, PUERTO RIC...",OFF,E BOUND,1143L,SAN JUAN CERAP,"LA ROMANA, DOMINICAN REPUBLIC (MDLR) - SJU",10 S MAYAGUEZ (TJMZ),"AT 9,000 FEET",41.119329,-95.908517
322,2024-03-31,GEORGIA,ATLANTA,"PRELIM INFO FROM FAA OPS: ATLANTA, GA/UAS SIGH...",MYR,N BOUND,1210E,ATLANTA TRACON,MYR - ATL,20 SE ATL,"AT 5,000 FEET",33.679741,-78.928321
323,2024-03-31,CALIFORNIA,FRESNO,"PRELIM INFO FROM FAA OPS: FRESNO, CA/UAS SIGHT...",FAT,SE BOUND,0920P,FRESNO ATCT,CCR - VIS,10 WNW FAT,"AT 8,000 FEET",36.776556,-119.718833
324,2024-03-31,NEW JERSEY,CALDWELL,"PRELIM INFO FROM FAA OPS: CALDWELL, NJ/UAS SIG...",CDW,NaN,1156E,MCGUIRE APCH,CDW - UNKN,NaN,NaN,40.875225,-74.28135


In [11]:
'''
Efficacy of algorithm --> only 4 values out of 326 yielded NaN for latitude and longitude. need to find ways to deal with edge case.
'''

merged_drone_df[merged_drone_df["latitude"].isna()]

,Date,State,City,Summary,code,Direction,Time,Authority Advised,Route,Location,Altitude,latitude,longitude
15,2024-01-02,COLORADO,COLORADO SPRINGS,"PRELIM INFO FROM FAA OPS: COLORADO SPRINGS, CO...",None,NaN,1217M,COLORADO SPRINGS ATCT,NaN,NaN,NaN,NaN,NaN
126,2024-02-14,FLORIDA,MILTON,"PRELIM INFO FROM FAA OPS: MILTON, FL/UAS SIGHT...",None,N BOUND,1230C,PENSACOLA TRACON,NaN,NaN,"AT 1,000 FEET",NaN,NaN
140,2024-02-16,TEXAS,BEAUMONT/PORT ARTHUR,PRELIM INFO FROM FAA OPS: BEAUMONT/PORT ARTHUR...,None,W BOUND,1328C,HOUSTON TRACON,NaN,NaN,"AT 2,800 FEET",NaN,NaN
166,2024-02-21,FLORIDA,SARASOTA,"PRELIM INFO FROM FAA OPS: SARASOTA, FL/UAS SIG...",None,E BOUND,1458E,MIAMI ARTCC,NaN,NaN,"AT 1,000 FEET",NaN,NaN


***Final merged_drone_df prior to 3D coordinate calculations, with latitude/longitude extracted from every row except for 4.***

In [12]:
merged_drone_df.head(4)

,Date,State,City,Summary,code,Direction,Time,Authority Advised,Route,Location,Altitude,latitude,longitude
0,2024-01-01,FLORIDA,MIAMI,"PRELIM INFO FROM FAA OPS: MIAMI, FL/UAS SIGHTI...",MIA,W BOUND,0001E,MIAMI ATCT,"PANAMA CITY, PANAMA (MPTO) - MIA",1 E MIA,AT 500 FEET,25.795361,-80.290116
1,2024-01-01,TEXAS,FORT WORTH,"PRELIM INFO FROM FAA OPS: FORT WORTH, TX/UAS S...",FTW,ESE BOUND,0015C,MEACHAM ATCT,FTW - FTW,8 ESE FTW,"AT 1,900 FEET",32.81977,-97.36245
2,2024-01-01,THE BAHAMAS,BAHAMAS,"PRELIM INFO FROM FAA OPS: BIMINI, BAHAMAS/UAS ...",FLL,E BOUND,0655E,MIAMI ARTCC,"FLL - GOVERNORS HARBOUR, BAHAMAS (MYEN)",7 E BIMINI VORTAC,"AT 9,000 FEET",26.071667,-80.149694
3,2024-01-01,FLORIDA,MIAMI,"PRELIM INFO FROM FAA OPS: MIAMI, FL/UAS SIGHTI...",HPN,NaN,1010E,MIA,MIA - HPN,4 W MIA,NaN,41.066953,-73.707566


In [41]:
''' 
EXTRACTING 3D COORDINATE OF DRONE BASED ON ALTITUDE, LONGITUDE, AND LATITUDE
Reference: 
    https://stackoverflow.com/questions/16614057/longitude-latitude-altitude-to-3d-cartesian-coordinate-systems
    for calculation of 3D coordinate
'''

# converting latitude and longitude columns to numeric values (currently an object)
merged_drone_df['latitude'] = pd.to_numeric(merged_drone_df['latitude'])
merged_drone_df['longitude'] = pd.to_numeric(merged_drone_df['longitude'])

# dropping rows with NaN values in latitude, longitude, or altitude
merged_drone_df = merged_drone_df.dropna(subset=['latitude', 'longitude', 'Altitude'])

# adjust altitude column with regex to delete the "AT" and "FEET" and only displays numbers
# convert the feet to meters since axis is in meters
merged_drone_df['altitude_numeric'] = merged_drone_df['Altitude'].str.extract(r'(\d{1,3}(?:,\d{3})*)')
merged_drone_df['altitude_numeric'] = merged_drone_df['altitude_numeric'].str.replace(',', '').astype(float)
merged_drone_df['alt_meters'] = merged_drone_df['altitude_numeric'] * 0.3048

# 3D coordinate extracting:

axis = 6378137
eccentricity = 8.1819190842622e-2
asq = np.power(axis, 2)
esq = np.power(eccentricity, 2)

def convert(df):
    df['lat_radian'] = np.radians(df['latitude'])
    df['long_radian'] = np.radians(df['longitude'])

    df['N'] = axis / np.sqrt(1 - esq * np.power(np.sin(df['lat_radian']), 2))

    df['x'] = (df['N'] + df['alt_meters']) * np.cos(df['lat_radian']) * np.cos(df['long_radian'])
    df['y'] = (df['N'] + df['alt_meters']) * np.cos(df['lat_radian']) * np.sin(df['long_radian'])
    df['z'] = ((1 - esq) * df['N'] + df['alt_meters']) * np.sin(df['lat_radian'])

    df.drop(columns=['lat_radian', 'long_radian', 'N'], inplace=True)

    return df

merged_drone_df2 = convert(merged_drone_df)
merged_drone_df2.head()

,Date,State,City,Summary,code,Direction,Time,Authority Advised,Route,Location,Altitude,latitude,longitude,altitude_numeric,alt_meters,x,y,z
0,2024-01-01,FLORIDA,MIAMI,"PRELIM INFO FROM FAA OPS: MIAMI, FL/UAS SIGHTI...",MIA,W BOUND,0001E,MIAMI ATCT,"PANAMA CITY, PANAMA (MPTO) - MIA",1 E MIA,AT 500 FEET,25.795361,-80.290116,500.0,152.40,969178.045641,-5.664042e+06,2.758734e+06
1,2024-01-01,TEXAS,FORT WORTH,"PRELIM INFO FROM FAA OPS: FORT WORTH, TX/UAS S...",FTW,ESE BOUND,0015C,MEACHAM ATCT,FTW - FTW,8 ESE FTW,"AT 1,900 FEET",32.819770,-97.362450,1900.0,579.12,-687606.695253,-5.321582e+06,3.437492e+06
2,2024-01-01,THE BAHAMAS,BAHAMAS,"PRELIM INFO FROM FAA OPS: BIMINI, BAHAMAS/UAS ...",FLL,E BOUND,0655E,MIAMI ARTCC,"FLL - GOVERNORS HARBOUR, BAHAMAS (MYEN)",7 E BIMINI VORTAC,"AT 9,000 FEET",26.071667,-80.149694,9000.0,2743.20,981164.522069,-5.650752e+06,2.787402e+06
7,2024-01-01,GEORGIA,ATLANTA,"PRELIM INFO FROM FAA OPS: ATLANTA, GA/UAS SIGH...",SBN,NaN,1444E,ATLANTA TRACON,SBN - ATL,8 NE ATL,"AT 8,000 FEET",41.708224,-86.317337,8000.0,2438.40,306407.167981,-4.760592e+06,4.223087e+06
8,2024-01-01,FLORIDA,MIAMI,"PRELIM INFO FROM FAA OPS: MIAMI, FL/UAS SIGHTI...",IAH,E BOUND,1714E,MIAMI ARTCC,IAH - MIA,1 SW MIAMI,AT 800 FEET,29.984435,-95.341442,800.0,243.84,-514729.667765,-5.505320e+06,3.169001e+06


In [42]:
'''
Final dataframe with drone sighting 3D Coordinates
'''

merged_drone_df2.head()

,Date,State,City,Summary,code,Direction,Time,Authority Advised,Route,Location,Altitude,latitude,longitude,altitude_numeric,alt_meters,x,y,z
0,2024-01-01,FLORIDA,MIAMI,"PRELIM INFO FROM FAA OPS: MIAMI, FL/UAS SIGHTI...",MIA,W BOUND,0001E,MIAMI ATCT,"PANAMA CITY, PANAMA (MPTO) - MIA",1 E MIA,AT 500 FEET,25.795361,-80.290116,500.0,152.40,969178.045641,-5.664042e+06,2.758734e+06
1,2024-01-01,TEXAS,FORT WORTH,"PRELIM INFO FROM FAA OPS: FORT WORTH, TX/UAS S...",FTW,ESE BOUND,0015C,MEACHAM ATCT,FTW - FTW,8 ESE FTW,"AT 1,900 FEET",32.819770,-97.362450,1900.0,579.12,-687606.695253,-5.321582e+06,3.437492e+06
2,2024-01-01,THE BAHAMAS,BAHAMAS,"PRELIM INFO FROM FAA OPS: BIMINI, BAHAMAS/UAS ...",FLL,E BOUND,0655E,MIAMI ARTCC,"FLL - GOVERNORS HARBOUR, BAHAMAS (MYEN)",7 E BIMINI VORTAC,"AT 9,000 FEET",26.071667,-80.149694,9000.0,2743.20,981164.522069,-5.650752e+06,2.787402e+06
7,2024-01-01,GEORGIA,ATLANTA,"PRELIM INFO FROM FAA OPS: ATLANTA, GA/UAS SIGH...",SBN,NaN,1444E,ATLANTA TRACON,SBN - ATL,8 NE ATL,"AT 8,000 FEET",41.708224,-86.317337,8000.0,2438.40,306407.167981,-4.760592e+06,4.223087e+06
8,2024-01-01,FLORIDA,MIAMI,"PRELIM INFO FROM FAA OPS: MIAMI, FL/UAS SIGHTI...",IAH,E BOUND,1714E,MIAMI ARTCC,IAH - MIA,1 SW MIAMI,AT 800 FEET,29.984435,-95.341442,800.0,243.84,-514729.667765,-5.505320e+06,3.169001e+06
